In [ ]:
from fastai.vision.all import *

path = untar_data(URLs.PETS)/'images'
images = get_image_files(path)
img = PILImage.create(images[0])
img.show()

def is_cat(x): return x[0].isupper() # label cats bsaed on filename rule providede by the dataset's creator
dls = ImageDataLoaders.from_name_func(path,
    get_image_files(path),
    valid_pct=0.2, # hold 20% of data and do not use it for training, VALIDATION SET
    seed=42,
    label_func=is_cat,
    item_tfms=Resize(224)) # item_tfms --> TRANSFORM, this one is applied to each item

learn = vision_learner(dls, resnet34, metrics=error_rate)
# resnet - architecture ResNet. 34 refers to the number of layers in this variant of the architecture
# METRIC - function that measures quality of the model's predictions using validation set
# error_rate - function provided by fastai which tells us percentage of the images that are being classified incorrectly
# Pretrained models -> Transfer learning

learn.fine_tune(1)



In [ ]:
from fastai.vision.all import *
import ipywidgets as widgets

btn_upload = widgets.FileUpload()
def on_upload_change(change):
    img = PILImage.create(btn_upload.value[0].content.tobytes())
    is_cat,_,probs=learn.predict(img)
    print(f"Is this a cat? {is_cat}")
    print(f"Probability it's a cat: {probs[1].item():.6f}")

btn_upload.observe(on_upload_change, names="value")
display(btn_upload)

In [ ]:
# %reset -f
from fastai.vision.all import *

path = untar_data(URLs.CAMVID_TINY)
print(path)
dls = SegmentationDataLoaders.from_label_func(
    path, 
    bs=8, 
    fnames= get_image_files(path/"images"),
        label_func=lambda o: path/'labels' /f'{o.stem}_P{o.suffix}',
        codes = np.loadtxt(path/'codes.txt', dtype=str)
    )
learn = unet_learner(dls, resnet34)
learn.fine_tune(8)

In [ ]:
learn.show_results(max_n=6, figsize=(7,8))

In [9]:
# Fix CUDA out of memory
import torch
import gc
import os
torch.cuda.empty_cache()
gc.collect()
print(torch.cuda.memory_summary(device=None, abbreviated=False))
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  16804 KiB |  18188 KiB |   2691 MiB |   2675 MiB |
|       from large pool |  16640 KiB |  16640 KiB |     16 MiB |      0 MiB |
|       from small pool |    164 KiB |   1548 KiB |   2675 MiB |   2675 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  16804 KiB |  18188 KiB |   2691 MiB |   2675 MiB |
|       from large pool |  16640 KiB |  16640 KiB |     16 MiB |

In [ ]:
from fastai.text.all import *

path = untar_data(URLs.IMDB)
dls = TextDataLoaders.from_folder(path, bs=8, valid='test')
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(2, 1e-2)
learn.predict("I really did not like that movie")

In [ ]:
doc(learn.predict)

In [4]:
from fastai.tabular.all import *
path = untar_data(URLs.ADULT_SAMPLE)

dls = TabularDataLoaders.from_csv(
    path/'adult.csv',
    path=path,
    y_names='salary',
    cat_name=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names=['age','fnlwgt','education-num'],
    procs=[Categorify, FillMissing, Normalize])

learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,0.449667,0.437388,0.791769,00:02
1,0.435458,0.431769,0.793458,00:02
2,0.448209,0.429795,0.795915,00:02


In [10]:
from fastai.collab import *

path = untar_data(URLs.ML_SAMPLE)
dls=CollabDataLoaders.from_csv(path/'ratings.csv')
learn=collab_learner(dls, y_range=(0.5, 5.5))
learn.fine_tune(10)
learn.show_results()

epoch,train_loss,valid_loss,time
0,1.500363,1.436336,00:00


epoch,train_loss,valid_loss,time
0,1.361482,1.375837,00:00
1,1.250112,1.194495,00:00
2,1.005154,0.899260,00:00
3,0.774453,0.769004,00:00
4,0.674206,0.731444,00:00
5,0.634348,0.717210,00:00
6,0.622144,0.710615,00:00
7,0.609659,0.707537,00:00
8,0.608234,0.706668,00:00
9,0.602466,0.706430,00:00


,userId,movieId,rating,rating_pred
0,84.0,28.0,4.0,3.192231
1,16.0,40.0,4.0,3.219264
2,60.0,82.0,5.0,4.417024
3,92.0,62.0,4.0,4.243615
4,68.0,89.0,4.0,3.952219
5,41.0,70.0,4.0,3.839693
6,44.0,21.0,3.5,3.699494
7,8.0,84.0,4.5,4.127815
8,92.0,47.0,4.5,4.491607
